1. [Задание 0. Ответ.](#task_0)
2. [Задание 1.](#task_1)   
    2.1 [Код функции.](#task_1.1)   
    2.2 [Пример различных весов.](#task_1.2)   
    2.3 [Предсказания.](#task_1.3)   
3. [Задание 2.](#task_3)   
    3.1 [Сравнение метрик.](#task_3.1)     
    3.2 [Ответ.](#task_3.2)     
4. [Задание 3.](#task_4)   
    4.1 [Про улучшение бейзлайнов на 5к](#task_4.1)   
    4.2 [Улучшение ItemItemRecomander через подбор соседей](#task_4.2)   
    4.3 [Aнсамблирование алгоритмов](#task_4.3)   
5. [Задание 4.](#task_5)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data["quantity_sales"] = data["quantity"]*data["sales_value"]

test_size_weeks = 3



data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

<a id='task_0'></a>
___ОТВЕТ___   
Когда мы взяли топ N самых популярных товаров. У нас могли появится пользователи, которые не покупали товар из этого топа. Для таких пользователей мы заводим фиктивный товар. В нашем случае 999999.    
Он нам нужем для того чтобы не потерять пользователей.   
Подобным методом мы ухудшаем качество рекомендаций. т.к. фековый товар может попасть в рекомендации пользователю. 

<a id='task_1'></a>
### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

<a id='task_1.1'></a>

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    items = np.array(items_weights['item_id'])
    prob = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=prob, replace=False)
    return recs.tolist()



<a id='task_1.2'></a>

In [5]:
def GetItemWeight(train):
    #Вес от частоты покупки
    item_weight1 = train.groupby('item_id')['user_id'].unique().reset_index()
    item_weight1['weight'] =  item_weight1['user_id'].apply(lambda x: len(x)) 
    del item_weight1['user_id']
    item_weight1['weight'] /= item_weight1.weight.sum()

    
    #На основе логарифма от цены. Здесь mean() намеренно
    item_weight2 = train.groupby('item_id')['sales_value'].mean().reset_index()
    item_weight2.sales_value *= 100000
    item_weight2.sales_value += 1
    item_weight2['weight'] = np.log(item_weight2['sales_value'])
    item_weight2.loc[np.isinf(item_weight2['weight'])] = 0
    item_weight2.weight /= item_weight2.weight.sum()
    del item_weight2['sales_value']
    

    #Вес от частоты покупки * на цену
    item_weight3 = train.groupby('item_id')['sales_value'].sum().reset_index()
    item_weight3['weight'] =  item_weight3['sales_value'] / item_weight3['sales_value'].sum()
    del item_weight3['sales_value']
    return item_weight1, item_weight2, item_weight3



In [6]:
item_weight1, item_weight2, item_weight3 = GetItemWeight(data_train)

In [7]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']  
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."
2,3,"[866211, 878996, 882830, 904360, 921345, 93194..."
3,4,"[836163, 857849, 877523, 878909, 883932, 89142..."
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69..."
5,6,"[840361, 851494, 851819, 851903, 863447, 87623..."
6,7,"[865569, 886703, 889731, 893400, 995436, 10205..."
7,8,"[824550, 824555, 826578, 835576, 841220, 84798..."
8,9,"[882190, 949294, 1023858, 1050429, 1070820, 92..."
9,10,"[959409, 1107661, 871377, 1025833, 8249164, 92..."


Сделайте предсказания
<a id='task_1.3'></a>

In [8]:
%%time
result['weighted_random1'] = result['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight1))
result['weighted_random2'] = result['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight2))
result['weighted_random3'] = result['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight3))
result.head(2)

CPU times: user 6.69 s, sys: 0 ns, total: 6.69 s
Wall time: 6.69 s


,user_id,actual,weighted_random1,weighted_random2,weighted_random3
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1103691, 7167967, 943699, 1104205, 7407975]","[975134, 1793277, 13682143, 8204833, 960631]","[961747, 1072659, 6919082, 1106523, 1138467]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[845196, 1126808, 13945141, 13213404, 890010]","[6608127, 1021519, 71139, 2017531, 888820]","[1111035, 844179, 887474, 956599, 1099834]"


<a id='task_3'></a>
### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [9]:
import numpy as np
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def popularity_recommendation(data, n=5):
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    return recs.tolist()

def random_recommendation(items, n=5):
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    return recs.tolist()




***

In [10]:
popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

***

In [11]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [13]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [14]:
result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

***

In [15]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.21 s, sys: 182 µs, total: 1.21 s
Wall time: 560 ms


In [16]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 67.2 ms, sys: 104 µs, total: 67.3 ms
Wall time: 66.5 ms


<a id='task_3.1'></a>

In [17]:
def GetMertics(data_result): 
    d = {}
    d['random'] = data_result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()
    d['weighted_random1'] = data_result.apply(lambda x: precision_at_k(x['weighted_random1'], x['actual'],  5), axis=1).mean() 
    d['weighted_random2'] = data_result.apply(lambda x: precision_at_k(x['weighted_random2'], x['actual'],  5), axis=1).mean() 
    d['weighted_random3'] = data_result.apply(lambda x: precision_at_k(x['weighted_random3'], x['actual'],  5), axis=1).mean() 
    d['popular'] = data_result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()
    d['itemitem'] = data_result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()
    d['cosine'] = data_result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()

    metrics = pd.DataFrame(d.items(), columns=['algorithm', 'value'])
    return metrics

GetMertics(result)

,algorithm,value
0,random,0.006483
1,weighted_random1,0.059784
2,weighted_random2,0.005122
3,weighted_random3,0.100520
4,popular,0.393037
5,itemitem,0.543737
6,cosine,0.575830


<a id='task_3.2'></a>
__Ответ__   


Среди весов немного выравлся вперед вес на основе умножения цены на количество купленного товара. Это связано с тем, что введением признака ___quantity_sales___ удалось получить небольшое обобщение. В следствии чего, веса распределились чуть лучше. Но всё равно плохо. Рандом он и есть рандом. 
Очень неудобным оказался вес на основе логорифма. Есть шанс получить отрицательный логарифм. Поэтому пришлось умножать на большое число и прибалять единицу. Ну и метрика показывает плохой результат на этом весе.

CosineRecommender и itemitem показывают гараздо лучшее качество. Здесь мы уже не опираемся на случайность. А пытаемся выявить закономертности между товарами. 

<a id='task_4'></a>
### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [18]:
print(data_train.item_id.unique().shape)

(5001,)


In [19]:
item_weight1, item_weight2, item_weight3 = GetItemWeight(data_train)

In [20]:
%%time
result5k = data_train.groupby('user_id')['item_id'].unique().reset_index()
result5k.columns=['user_id', 'actual']
result5k['actual'] = result5k['actual'].apply(lambda x: list(x))
items = data_train.item_id.unique()
pops = popularity_recommendation(data_train, n=5)
result5k['popular_recommendation'] = result5k['user_id'].apply(lambda x: pops)
result5k['random_recommendation'] = result5k['user_id'].apply(lambda x: random_recommendation(items, n=5))
result5k['weighted_random1'] = result5k['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight1))
result5k['weighted_random2'] = result5k['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight2))
result5k['weighted_random3'] = result5k['user_id'].apply(lambda x:  weighted_random_recommendation(item_weight3))

result5k['itemitem'] = result5k['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

result5k['cosine'] = result5k['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 1.72 s, sys: 3.95 ms, total: 1.72 s
Wall time: 1.72 s


In [21]:
GetMertics(result5k)

,algorithm,value
0,random,0.056823
1,weighted_random1,0.108283
2,weighted_random2,0.055062
3,weighted_random3,0.318207
4,popular,0.431693
5,itemitem,0.775510
6,cosine,0.775510


<a id='task_4.1'></a>
Кажется что на 5000 товарах качество возрасло. Но это происходит из-за введения фейкового товара. Мы просто чаще угадываем товар 999999. И он же теперь и самый популярный.


<a id='task_4.2'></a>
__Улучшение ItemItemRecommender, выбирая число соседей 𝐾.__

In [22]:
%%time
scores = {}
for i in range(1,50):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)
    result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    scores[str(i)] = result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()
    

CPU times: user 1min 9s, sys: 59.8 ms, total: 1min 9s
Wall time: 36.5 s


In [23]:
metrics = pd.DataFrame(scores.items(), columns=['Ki', 'socre'])
metrics

,Ki,socre
0,1,0.792063
1,2,0.757276
2,3,0.683373
3,4,0.577751
4,5,0.543737
5,6,0.543257
6,7,0.537255
7,8,0.539656
8,9,0.542217
9,10,0.546218


__Вывод__   
Ожидаемо, что чем меньше соседей тем лучше прогноз.   
Кефир похож на ряженку и снежок. На кого-то больше-на кого-то меньше. Так не надо рекомендовать эти два товара сразу. А лучше один наиболее похожий.   
Хотя конечно, удивительно что прогноз вырос так сильно.


<a id='task_4.3'></a>
__Aнсамблирование алгоритмов__

In [24]:
def GetUnion():
    r = set(random_recommendation(items, n=5))
    w1 = set(weighted_random_recommendation(item_weight1))
    w2 = set(weighted_random_recommendation(item_weight2))
    w3 = set(weighted_random_recommendation(item_weight3))
    res = r&w1&w2&w3 
    while len(res) < 5:
        res.add((res^r^w1^w2^w3).pop()) 
    return list(res)
GetUnion()    

result['ansamble'] = result['user_id'].apply(lambda x:  GetUnion())    
score = result.apply(lambda x: precision_at_k(x['ansamble'], x['actual'],  5), axis=1).mean()         
print(score)

0.08387354941976792


__Вывод__  
Получили некоторое усредненние по ансамблю.

<a id='task_5'></a>
### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?   
__Ответ__    
$ \bar{r_u}$ - средняя оценка пользователя u по всем оценённым товарам.   
$ \mu $ - среднее по всем оценкам.



Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

In [25]:
def CosineSimilarity(a,b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [26]:
def GetMuRUUserValue(user_id, user_item_matrix):
    ru = user_item_matrix.loc[user_id].sum()/user_item_matrix.shape[1] 
    mu = 0
    user_value = np.array(user_item_matrix.loc[user_id]) 
   
    for i in range(user_item_matrix.shape[0]):
        mu += user_item_matrix.iloc[i].sum()/user_item_matrix.shape[1]
    mu /= user_item_matrix.shape[0]
    return ru, mu, user_value

def UserUserRecommender(user_id, item_id, user_item_matrix, ru, mu, user_value):
    sim = 0
    res = 0
    um = user_value
    for i in range(user_item_matrix.shape[0]):
        s  = CosineSimilarity(um,np.array(user_item_matrix.iloc[i]))
        sim += s
        res += s*(user_item_matrix.iloc[i].loc[item_id] - user_item_matrix.iloc[i].sum()/user_item_matrix.shape[1] - mu)
    return res/sim + mu + ru
                                
    
        

In [27]:
ru, mu, user_value = GetMuRUUserValue(6, user_item_matrix)

In [28]:
%%time
UserUserRecommender(6, 818980, user_item_matrix, ru, mu, user_value)

CPU times: user 925 ms, sys: 0 ns, total: 925 ms
Wall time: 924 ms


0.03620843339195714

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.